# 1. Preprocessing

In [ ]:
# Converting resn(residue number) in all ligand files to distinct values

In [ ]:
import glob

# Creating a list containing the files' names
list_names = []
for name in glob.glob('/home/vincent/Downloads/gcc_test/*.mol2'):
    list_names.append(name)
list_names.sort()


def change_func(filename,num):
  # Read in the file
  with open(f'{filename}', 'r') as file :
    filedata = file.read()
  # Replace the target string
  filedata = filedata.replace('<0>', f'{num}')
  # Write the file out again
  with open(f'{filename}', 'w') as file:
    file.write(filedata)

In [ ]:
j = 1
for i in list_names:
  change_func(i,j)
  j += 1

# 2. Loading all files into the PyMol

In [ ]:
from pymol import cmd
import glob
import pandas as pd

In [ ]:
list_ligands = []
for name in glob.glob('/home/vincent/wee1/*.mol2'):
    list_ligands.append(name)

for i in list_ligands:
    cmd.load(i)

# 3. Object creation in PyMol for acceptor and donor atoms

In [ ]:
# Initialize by adding hydrogens and removing solvents to prepare the structure
cmd.h_add()  # Correct usage to ensure the command is executed
cmd.remove("solvent")  # Removes all solvent molecules, typically water

# Define and create selections for donor and acceptor atoms within the molecule
cmd.create("acceptors_object", "elem O,F or (elem N and not (neighbor hydro) and not (fc. > 0))")
cmd.create("donors_object", "(elem O + elem N + elem S) and (neighbor hydrogens)")

# Dictionary to store atoms categorized as donors, acceptors, and from pharmacophore query
atom_categories = {"don": [], "acc": [], "pharmacophore": []}

# Populate the dictionary with atom details for each category
cmd.iterate('donors_object', "don.append('/%s/%s/%s/%s`%s/%s' % (model, segi, chain, resn, resi, name))", space=atom_categories)
cmd.iterate('acceptors_object', "acc.append('/%s/%s/%s/%s`%s/%s' % (model, segi, chain, resn, resi, name))", space=atom_categories)

# Load the pharmacophore model and add its atom details to the query category
pharmacophore_name = "wee1_bbr_3CFF_query"
cmd.load(f"/home/vincent/wee1/{pharmacophore_name}.pdb", "ppara")
cmd.iterate("ppara", "pharmacophore.append('/%s/%s/%s/%s`%s/%s' % (model, segi, chain, resn, resi, name))", space=atom_categories)

# 4. Defining primary functions for distance measurements

In [ ]:
# Define functions to measure distances for potential interactions between atoms
def pharmacopcount_donor_interactionshore(atom):
    """Calculate and return the number of donor interactions within 2 angstroms for a given atom."""
    count = 0
    for i in atom_categories["don"]:
        if cmd.get_distance(i, atom) <= 2:
            count += 1
    return count

def count_acceptor_interactions(atom):
    """Calculate and return the number of acceptor interactions within 2 angstroms for a given atom."""
    count = 0
    for i in atom_categories["acc"]:
        if cmd.get_distance(i, atom) <= 2:
            count += 1
    return count

def count_hydrophobic_interactions(atom):
    """Identify and count hydrophobic interactions within 3 angstroms for a given atom."""
    cmd.create("hydrophobic", f"((elem C and not (neighbor elem N) and not (neighbor elem O) and not (neighbor elem F) and not acceptors_object and not donors_object and not ppara)) within 3 of ({atom})")
    temp = {"hydrophobic": []}
    cmd.iterate("hydrophobic", "hydrophobic.append('%s' % (name))", space=temp)
    cmd.delete("hydrophobic")
    return len(temp["hydrophobic"])

# 5. Obtaining interaction summary

In [ ]:
# Compile interactions data for each atom in the query
interaction_summary = []
for at in atom_categories["pharmacophore"]:
    interactions = [at]
    # Determine the role and interactions based on atom type and properties
    if at[8] == "G":
        interactions += ["Grease", "NA", "NA", count_hydrophobic_interactions(at)]
    elif at[8] == "B":
        if at[-1] == "O":
            interactions += ["Donor", count_acceptor_interactions(at), pharmacopcount_donor_interactionshore(at), "NA"]
        else:
            interactions += ["Acceptor", count_acceptor_interactions(at), pharmacopcount_donor_interactionshore(at), "NA"]
    else:
        role = "Donor" if at[8] == "D" else "Acceptor"
        interactions += [role, count_acceptor_interactions(at), pharmacopcount_donor_interactionshore(at), "NA"]
    interaction_summary.append(interactions)



# 6. Dataframe creation

In [ ]:
# Create a DataFrame for analysis and visualization
table_labels = ["Atom Details", "Label", "Acceptor H-Bonds", "Donor H-Bonds", "Hydrophobic Interactions"]
table_index = [f"{pharmacophore_name[0:4].capitalize()}_atom_{num}" for num in range(1, len(interaction_summary)+1)]
df = pd.DataFrame(data=interaction_summary, index=table_index, columns=table_labels)

In [ ]:
df.to_csv("/home/vincent/wee1/interaction_summary_wee1.csv")